# Introducción

El objectivo de este práctico es explorar como la topología de la red de contactos (grafo de potenciales contagios entre individuos) puede impactar en el comportamiento de una epidemia.

Para esto, vamos a implementar un simulador que nos ayude a corroborar hipótesis acerca de la difusión de procesos dinámicos en grafos. 

In [1]:
%matplotlib inline

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import operator
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed, IntSlider
from matplotlib.patches import Patch

plt.rcParams["figure.figsize"] = (12, 8)

In [3]:
STATE = "state"
STATE_HISTORY = "state_history"

SUSCEPTIBLE = 0
INFECTED = 1
RECOVERED = 2

STATE_COLORS = {
    SUSCEPTIBLE: "blue",
    INFECTED: "red",
    RECOVERED: "green"
    
}

SEED = 42

In [4]:
def SIR_node_update(cur_state, neig_states, random_state, beta, gamma, **kwargs):
    """
    This function implementsthe SIR model node update. It receives the
    state of a node, the state of all nodes surrounding it and calculates
    the probabilistic node update based on the SIR model formulation.
    
    Paramteres
    ----------
    cur_state: int
        The state of the node for which the update will be calculated
    neig_states: list of ints
        The state of all the neighbours of the current state.
    random_state: np.random.RandomState
        This is the random state generator object. Use it to generate
        any random number required within this function. This gurantees
        a consisten output.
    beta: float
        The beta paramteter from the SIR model
    gamma: float
        The gamma paramters from the SIR model
        
    """
    
    ### START CODE HERE
    if len(neig_states) > 0:
        infected_neigs = sum([1 if s == INFECTED else 0 for s in neig_states])
    else:
        infected_neigs = 0
    
    prob_sus_to_inf = beta * infected_neigs #TODO: esto está bien? puede dar > 1
    if prob_sus_to_inf>1:
      prob_sus_to_inf = 1
    
    new_state = cur_state
    if cur_state == SUSCEPTIBLE and prob_sus_to_inf > 0:
        if random_state.binomial(1, p=prob_sus_to_inf) == 1:
            new_state = INFECTED
    if cur_state == INFECTED:
        if random_state.binomial(1, p=gamma) == 1:
            new_state = RECOVERED

    ### END CODE HERE
    
    return new_state


In [5]:
def simulate_difussion(G, initial_states, random_state, update_func = None, **kwargs):

    if update_func is None:
        update_func = SIR_node_update
    
    [G.nodes[k].update({STATE: v}) for k, v in initial_states.items()]
    [G.nodes[k].update({STATE_HISTORY: []}) for k, v in initial_states.items()]

    T = 1
    while True:

        new_states = dict()
        for cur in G.nodes():
            neigs = list(G.neighbors(cur))
            state_cur = G.nodes[cur][STATE]
            state_neigs = [G.nodes[n][STATE] for n in neigs]

            new_state = update_func(state_cur, state_neigs, random_state, **kwargs)
            new_states[cur] = new_state

        for cur in G.nodes():

            old_state = G.nodes[cur][STATE]
            new_state = new_states[cur]

            G.nodes[cur][STATE_HISTORY].append(old_state)
            G.nodes[cur].update({STATE: new_state})
        
        if sum(v == INFECTED for v in new_states.values()) == 0:
            break
        else:
            T += 1
    
    T += 1
    for n in G.nodes(): G.nodes[n][STATE_HISTORY].append(G.nodes[n][STATE])
    
    node_colors = dict(
    (t, [STATE_COLORS[v[STATE_HISTORY][t]] for k, v in G.nodes(data=True)]) for t in range(T)
    )
    return node_colors, T

In [6]:
def plot_difussion(t, G, colors, layout):
    
    legend_elements = [
        Patch(facecolor=STATE_COLORS[SUSCEPTIBLE], edgecolor='k', label="Susceptible"),
        Patch(facecolor=STATE_COLORS[INFECTED], edgecolor='k', label='Infected'),
        Patch(facecolor=STATE_COLORS[RECOVERED], edgecolor='k', label='Recovered')
    ]

    nx.draw_networkx_nodes(G, pos=layout, node_color=node_colors[t])
    _ = nx.draw_networkx_labels(G, pos=layout)
    nx.draw_networkx_edges(G, pos=layout)
    
    plt.legend(handles=legend_elements, loc="upper center", bbox_to_anchor=(0.5, 1.1), ncol=3)
    plt.title(f"Timeslot: {t}")
    plt.show()

In [7]:
def stats_difussion(G, view=True):

  t_range = len(G.nodes[0][STATE_HISTORY])
  susceptible = [sum([v[STATE_HISTORY][t] == SUSCEPTIBLE for k, v in G.nodes(data=True)]) for t in range(t_range)]
  infected = [sum([v[STATE_HISTORY][t] == INFECTED for k, v in G.nodes(data=True)]) for t in range(t_range)]
  recovered = [sum([v[STATE_HISTORY][t] == RECOVERED for k, v in G.nodes(data=True)]) for t in range(t_range)]

  if view:
    plt.plot(range(t_range), susceptible, color=STATE_COLORS[SUSCEPTIBLE], linestyle = 'dashed', alpha=0.3)
    plt.plot(range(t_range), infected, color=STATE_COLORS[INFECTED])        
    plt.plot(range(t_range), recovered, color=STATE_COLORS[RECOVERED], linestyle = 'dashed', alpha=0.3)
    legend_elements = [
          Patch(facecolor=STATE_COLORS[SUSCEPTIBLE], edgecolor='k', label="Susceptible"),
          Patch(facecolor=STATE_COLORS[INFECTED], edgecolor='k', label='Infected'),
          Patch(facecolor=STATE_COLORS[RECOVERED], edgecolor='k', label='Recovered')
      ]
    plt.legend(handles=legend_elements, loc="upper center", bbox_to_anchor=(0.5, 1.1), ncol=3)
    plt.title("Epidemic evolution")
    plt.xlabel("Time")
    plt.ylabel("# Population")
    plt.show()

  return susceptible,infected,recovered


In [8]:
def stats_trials(infected_trials, title='Epidemic evolution', view=True):
  
  t_range = max([len(i) for i in infected_trials])
  sum_trials = np.zeros((t_range))
  len_trials = np.zeros((t_range))
  for i in infected_trials:
    i = np.array(i)

    l = np.ones(i.shape)
    l.resize(sum_trials.shape)
    len_trials = len_trials + l

    i.resize(sum_trials.shape)
    sum_trials = sum_trials + i

  infected_mean = sum_trials/len_trials

  if view:
    for i in range(len(infected_trials)):
        plt.plot(range(len(infected_trials[i])), infected_trials[i], color=STATE_COLORS[INFECTED], alpha=0.1)       

    plt.plot(range(len(infected_mean)), infected_mean, color=STATE_COLORS[INFECTED])       
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("# Population")
    plt.show()

  return infected_mean

# 1) Fundamentos en los procesos dinámicos de epidemias


Seguir la Sección 8.5 del libro [SANDR], entendiendo los procesos epidémicos tradicionales del tipo SIR.

#2) Primer ejemplo, difusión en grafos lineales (caminos)

Para empezar a explorar el primer modelo de difusión vamos a trabajar con uno de los grafos más simples de todos: el camino.

El grafo camino es interesante desde el punto de vista de una epidemia, dado que una vez que un nodo se recupera, este impide la difusión del grafo entre los dos grupos de nodos a sus lados.

Para empezar vamos a generar un grafo.

In [9]:
N=10
G = nx.path_graph(N)
initial_states = dict((n, SUSCEPTIBLE) for n in G.nodes())
initial_states[1] = INFECTED
layout = nx.spring_layout(G)

In [10]:
r = np.random.RandomState(SEED - 1)
node_colors, total_timeslots = simulate_difussion(
    G,
    initial_states,
    r,
    SIR_node_update,
    beta=0.25,
    gamma=0.1
)

In [11]:
len(G.nodes(data=True)[8][STATE_HISTORY])

Es posible ver la difusión de la epidemia, moviendose en el tiempo utilizando la barra de la siguiente figura.

In [12]:
interact(
    plot_difussion,
    t=IntSlider(value=0, min=0, max=total_timeslots - 1),
    G=fixed(G),
    colors=fixed(node_colors),
    layout=fixed(layout)
)

También podemos ver como evoluciona la cantidad de infectados.

In [13]:
_,_,_ = stats_difussion(G)

#3) Difusión en grafo Erdös-Renyi

Como segundo ejemplo de difusión vamos a considerar el caso de un grafo aleatorio Erdös-Renyi

In [14]:
N, p = 28, 0.3
G = nx.erdos_renyi_graph(n=N, p=p, seed=SEED)
initial_states = dict((n, SUSCEPTIBLE) for n in G.nodes())
initial_states[1] = INFECTED
layout = nx.spring_layout(G)

In [15]:
r = np.random.RandomState(SEED)
node_colors, total_timeslots = simulate_difussion(
    G,
    initial_states,
    r,
    SIR_node_update,
    beta=0.1,
    gamma=0.1
)

In [16]:
interact(
    plot_difussion,
    t=IntSlider(value=0, min=0, max=total_timeslots - 1),
    G=fixed(G),
    colors=fixed(node_colors),
    layout=fixed(layout)
)
_,_,_ = stats_difussion(G)

##3.1) ¿Qué puede decir de la diferencia de velocidad con la que el proceso se esparse a través de la red para el caso del grafo camino y para el grafo Erdös-Renyi?

In [17]:
### START CODE HERE
### END CODE HERE


# 4) Difusión en grafos de distintos modelos

Compararemos la difusión de la epidemia para los modelos:

* Erdös-Renyi
* Barabasi Albert
* Watts Strogatz

Para ver estadísticamente el resultado, realizaremos 20 pruebas por modelo, donde sortearemos la toppología y el vértice inicial.

Para ser justos, intentaremos que todos los grafos tengan 250 vértices y 1250 aristias (aproximadamente).


In [18]:
#parámetros en común

N = 250 #vertices

#epidemia
beta=0.5/20
gamma=1.0/20

trials = 20 #pruebas

#mismo estado inicial para todos los grafos
initial_states = dict((n, SUSCEPTIBLE) for n in range(N))
initial_infected = np.random.randint(0,N-1, size=1)[0]
#initial_states[1] = INFECTED
initial_states[initial_infected] = INFECTED

Simular Erdös-Renyi

In [19]:
p = 0.041
infected_trials = []
for i in range(trials):
  G = nx.erdos_renyi_graph(n=N, p=p)

  node_colors, total_timeslots = simulate_difussion(
      G,
      initial_states,
      np.random.RandomState(SEED),
      SIR_node_update,
      beta=beta,
      gamma=gamma
  )

  print(G.number_of_edges())

  _, infected_i,_ = stats_difussion(G, view=False)
  infected_trials.append(infected_i)

infected_mean_erdosrenyi = stats_trials(infected_trials, title="Epidemic evolution: Erdös-Renyi")

Simular Barabasi Albert

In [20]:
### START CODE HERE
### END CODE HERE

infected_mean_barabasi = stats_trials(infected_trials, title="Epidemic evolution: Barabasi Albert")

Simular Watts Strogatz

In [ ]:
### START CODE HERE
### END CODE HERE

infected_mean_ws = stats_trials(infected_trials, title="Epidemic evolution: Watts Strogatz")

Resultado de la comparación

In [ ]:
plt.plot(infected_mean_erdosrenyi, color='red')
plt.plot(infected_mean_barabasi, color='green')
plt.plot(infected_mean_ws, color='blue')
plt.title("Evolución promedio de los distintos modelos")
plt.xlabel("Time")
plt.ylabel("# Population")
legend_elements = [
          Patch(facecolor='red', edgecolor='k', label="Erdos Renyi"),
          Patch(facecolor='green', edgecolor='k', label='Barabasi Albert'),
          Patch(facecolor='blue', edgecolor='k', label='Watts Strogatz')
      ]
plt.legend(handles=legend_elements, loc="upper center", bbox_to_anchor=(0.5, 1.1), ncol=3)
plt.show()


#5) Limitando el número máximo de infectados

En esta sección intentaremos encontrar buenas estrategias para limitar y disminuir la máxima cantidad de nodos infectados a lo largo del proceso.

Para esto vamos a permitirnos `inmunizar` a dos nodos (asignarles el estado inicial `RECUPERADO`) y evaluar el efecto de tal política en la epidemia.

Para estos experimentos vamos a usar el [grafo de Tutte](https://en.wikipedia.org/wiki/Tutte_graph)

Para simplificar la experimentación, vamos a asumir que la epidemia comienza en el nodo `3`.

In [ ]:
G = nx.tutte_graph()

layout = nx.spring_layout(G, iterations=300, seed=SEED)

initial_states = dict((n, SUSCEPTIBLE) for n in G.nodes())
initial_states[3] = INFECTED

In [ ]:
nx.draw_networkx(G, pos=layout)

In [ ]:
r = np.random.RandomState(SEED)
node_colors, total_timeslots = simulate_difussion(
    G,
    initial_states,
    r,
    SIR_node_update,
    beta=0.3,
    gamma=0.05
)

In [ ]:
interact(
    plot_difussion,
    t=IntSlider(value=0, min=0, max=total_timeslots - 1),
    G=fixed(G),
    colors=fixed(node_colors),
    layout=fixed(layout)
)

In [ ]:
def maximum_infected(G):
    """
    Calculates the total maximum of infected nodes
    in any given timeslot.
    
    Parameters:
    ------------
    G: nx.Graph
        The graph of the network. Assume that the epidemic has
        already been simulated and that each node in the graph
        contains the attribute `STATE_HISTORY`
        
    Returns
    -------
    max_infected: int
        The maximum number of infected nodes in any timeslot.
    """
    
    ### START CODE HERE
    
    t_range = len(G.nodes[0][STATE_HISTORY])
    
    return max(sum([v[STATE_HISTORY][t] == INFECTED for k, v in G.nodes(data=True)]) for t in range(t_range))

    ### END CODE HERE
    

In [ ]:
assert maximum_infected(G) == 29 # Hay como máximo 29 infectados en la corrida por defecto

##5.1) Eligiendo el nodo para "inmunizar"

Vamos a comprar las siguientes estrategias de inmunización:

* Un vécino del primer nodo infectado (19)
* El nodo con mayor `betweeness centrality`
* Un nodo poco relacionado (nodo 37)

Todos los otros parámetros permaneceran iguales.

### Estratégia: Un vécino

In [ ]:
initial_states_copy = initial_states.copy()
initial_states_copy[19] = RECOVERED

In [ ]:
r = np.random.RandomState(SEED)
node_colors, total_timeslots = simulate_difussion(
    G,
    initial_states_copy,
    r,
    SIR_node_update,
    beta=0.3,
    gamma=0.05
)

In [ ]:
maximum_infected(G)

Logramos reducir la maxima cantidad de infectados en 1.

### Estrategia: Mayor `betweeness centrality`

In [ ]:
### START CODE HERE
### END CODE HERE


In [ ]:
initial_states_copy = initial_states.copy()
initial_states_copy[selected_node] = RECOVERED

In [ ]:
r = np.random.RandomState(SEED)
node_colors, total_timeslots = simulate_difussion(
    G,
    initial_states_copy,
    r,
    SIR_node_update,
    beta=0.3,
    gamma=0.05
)

In [ ]:
maximum_infected(G)

Logramos reducir la maxima cantidad de infectados en 5.

### Estrategia: Nodo poco relacionado (37)

In [ ]:
initial_states_copy = initial_states.copy()
initial_states_copy[37] = RECOVERED

In [ ]:
r = np.random.RandomState(SEED)
node_colors, total_timeslots = simulate_difussion(
    G,
    initial_states_copy,
    r,
    SIR_node_update,
    beta=0.3,
    gamma=0.05
)

In [ ]:
maximum_infected(G)

Logramos reducir la maxima cantidad de infectados en 2.